# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [64]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [65]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [66]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['', 'Logged In', 'Theodore', 'M', '0', 'Smith', '', 'free', 'Houston-The Woodlands-Sugar Land, TX', 'GET', 'Home', '1.54031E+12', '154', '', '200', '1.54129E+12', '52']


In [67]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [68]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [69]:
# TO-DO: Create a Keyspace 
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS ac_hw 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }'''
                   )

except Exception as e:
    print(e)

#### Set Keyspace

In [70]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('ac_hw')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [82]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query_1="SELECT artist, song, length FROM query1 WHERE  itemInSession='4' AND sessionId='338'"

#create table one
table_1='''CREATE TABLE IF NOT EXISTS query1(firstname text,lastname text,song text, 
artist text,length text,itemInSession text,sessionId text, PRIMARY KEY(itemInSession, sessionId))'''

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query_2="SELECT artist, song, firstname, lastname FROM query2 WHERE userid='10' AND  sessionId='182'"  

#create table two
table_2='''CREATE TABLE IF NOT EXISTS query2(firstname text,lastname text,
song text, artist text,length text,itemInSession text,sessionId text, userid text, PRIMARY KEY(userid, sessionId,iteminsession))'''

## query3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_3="SELECT firstname, lastname FROM query3 WHERE song='All Hands Against His Own'"
#create table three
table_3='CREATE TABLE IF NOT EXISTS query3(firstname text,lastname text,song text, PRIMARY KEY(song))'



In [72]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
session.execute(table_1)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO query1(firstname,lastname,song,artist,length,itemInSession, sessionId)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1],line[4],line[9],line[0],line[5],line[3],line[8]))

#### Do a SELECT to verify that the data have been inserted into each table

In [76]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
#query1
try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.artist,row.length)

Music Matters (Mark Knight Dub) Faithless 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [74]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
session.execute(table_2)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO query2(firstname,lastname,song,artist,length,itemInSession, sessionId,userid)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1],line[4],line[9],line[0],line[5],line[3],line[8],line[10]))
                    

In [75]:
## query2

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.firstname, row.lastname, row.song)

Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


In [79]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
session.execute(table_3)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO query3(firstname,lastname,song)"
        query = query + "VALUES(%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1],line[4],line[9]))
                    

In [83]:
## query3

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Tegan Levine


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [86]:
drop=2
tables=['query1','query2','query3']
query='DROP TABLE IF EXISTS '+tables[drop]
session.execute(query)

### Close the session and cluster connection¶

In [87]:
session.shutdown()
cluster.shutdown()